# SQL Murder Mystery

A crime has taken place and the detective needs your help. The detective gave
you the crime scene report, but you somehow lost it.

## Clue 1

You vaguely remember that the crime was a **murder** that occurred sometime on
**Jan. 15, 2018** and that it took place in **SQL City**. Start by retrieving
the corresponding crime scene report from the police department's database. If
you want to get the most out of this mystery, try to work through it only
using your SQL environment and refrain from using a notepad.

In [1]:
import sqlite3

import pandas as pd
from IPython.display import Markdown, display

In [2]:
conn = sqlite3.connect("sql-murder-mystery.db")

In [3]:
def lazy_avoid_typing_conn(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, conn)

In [4]:
lazy_avoid_typing_conn(
    """
SELECT "membership_id",
       "check_in_date",
       "check_in_time",
       "check_out_time"
FROM   "get_fit_now_check_in"
LIMIT  1000 """
)

,membership_id,check_in_date,check_in_time,check_out_time
0,NL318,20180212,329,365
1,NL318,20170811,469,920
2,NL318,20180429,506,554
3,NL318,20180128,124,759
4,NL318,20171027,418,1019
...,...,...,...,...
995,I0UIP,20170124,687,1011
996,I0UIP,20170323,670,1184
997,I0UIP,20170326,620,1131
998,I0UIP,20171027,121,368


![image.png](https://mystery.knightlab.com/schema.png)

In [5]:
lazy_avoid_typing_conn(
    '''SELECT "description"
FROM   "crime_scene_report"
WHERE  date = 20180115
       AND type = "murder"
       AND city = "SQL City"'''
)["description"].iloc[0]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

In [6]:
display(Markdown("## Clue 2"))
display(
    Markdown(
        lazy_avoid_typing_conn(
            '''SELECT "description"
FROM   "crime_scene_report"
WHERE  date = 20180115
       AND type = "murder"
       AND city = "SQL City"'''
        )["description"].iloc[0]
    )
)

## Clue 2

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".

In [7]:
lazy_avoid_typing_conn(
    '''SELECT
    "id", 
    "name", 
    "license_id", 
    "address_number", 
    "address_street_name", 
    "ssn",
    MAX("address_number") AS last_house
FROM "person"
WHERE "address_street_name" = "Northwestern Dr"'''
)

,id,name,license_id,address_number,address_street_name,ssn,last_house
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949,4919


In [8]:
lazy_avoid_typing_conn(
    '''SELECT
    "id", 
    "name", 
    "license_id", 
    "address_number", 
    "address_street_name", 
    "ssn"
FROM "person"
WHERE "address_street_name" = "Franklin Ave" AND "name" LIKE "Annabel%"'''
)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [9]:
display(Markdown("## Clue 3"))
for i in range(2):
    display(
        Markdown(
            lazy_avoid_typing_conn(
                """SELECT "person_id",
           "transcript"
    FROM   "interview"
    WHERE  person_id = 14887
            OR person_id = 16371"""
            )["transcript"].iloc[i]
        )
    )
    print()

## Clue 3

I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.

In [10]:
lazy_avoid_typing_conn(
    """SELECT *
FROM   get_fit_now_member
       INNER JOIN person
               ON get_fit_now_member.person_id = person.id
       INNER JOIN drivers_license
               ON person.license_id = drivers_license.id
WHERE  get_fit_now_member.membership_status = 'gold'
       AND get_fit_now_member.id LIKE '48Z%'
       AND drivers_license.plate_number LIKE '%H42W%' """
)

,id,person_id,name,membership_start_date,membership_status,id,name,license_id,address_number,address_street_name,ssn,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,48Z55,67318,Jeremy Bowers,20160101,gold,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


In [11]:
display(Markdown("## Clue 4"))
display(
    Markdown(
        lazy_avoid_typing_conn(
            """SELECT "transcript"
FROM   "interview"
WHERE  "person_id" = 67318;  """
        )["transcript"].iloc[0]
    )
)

## Clue 4

I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.


In [12]:
lazy_avoid_typing_conn('''SELECT *
FROM 
  drivers_license
INNER JOIN person
ON drivers_license.id = person.license_id
INNER JOIN facebook_event_checkin
ON person.id = facebook_event_checkin.person_id
WHERE 
  drivers_license.height BETWEEN 65 
  AND 67 
  AND drivers_license.car_make = "Tesla" 
  AND drivers_license.hair_color = "red" 
  AND drivers_license.gender = "female"
  AND event_name = "SQL Symphony Concert"
GROUP BY person.id''')

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn,person_id,event_id,event_name,date
0,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171206


In [13]:
lazy_avoid_typing_conn("SELECT * FROM person WHERE id = 99716")

,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [14]:
%%bash
sqlite3 sql-murder-mystery.db "INSERT INTO solution VALUES (1, 'Miranda Priestly');SELECT value FROM solution;"

Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!
